In [110]:
import numpy as np
import os
import glob
import nibabel as nib
from scipy.ndimage.interpolation import zoom
from scipy.ndimage.measurements import label
import matplotlib.pyplot as plt

In [111]:
study_folder = r'N:\2_Analysis\automated_analysis\Training Cases\FLINT_CYNCH_SEGSEG\flint_output'

In [99]:
def get_all_paths(study_folder):
    paths=[]
    for path in os.listdir(study_folder):
        rpath = os.path.join(study_folder, path)
        paths.append(rpath)
    return paths

def process_mask(arr, out_shape):

    liv = arr[:,:,:,0]
    liv = zoom(liv, (out_shape[0]/192, out_shape[1]/192, out_shape[2]/96))
    liv[liv >= 0.5] = 1; liv[liv < 0.5] = 0
    liv = largestConnectedComponent(liv)

    arr_resize = np.zeros((out_shape[0], out_shape[1], out_shape[2], 10))
    for ii in range(arr.shape[-1]):
        arr_resize[:,:,:,ii] = zoom(arr[:,:,:,ii], (out_shape[0]/192, out_shape[1]/192, out_shape[2]/96))

    arr_resize[arr_resize <= 0.5] = 0.
    arr = np.argmax(np.concatenate([ np.zeros((out_shape[0], out_shape[1], out_shape[2],1)), arr_resize[:,:,:,1:] ], -1), axis=-1)

    arr_resize = np.zeros((out_shape[0], out_shape[1], out_shape[2], 9))
    for ii in range(9):
        arr_resize[:,:,:,ii][np.where(arr == (ii+1))] = 1
        arr_resize[:,:,:,ii] = largestConnectedComponent(arr_resize[:,:,:,ii])

    arr = np.argmax(np.concatenate([ np.zeros((out_shape[0], out_shape[1], out_shape[2],1)), arr_resize ], -1), axis=-1)

    return liv,arr

def largestConnectedComponent(cnn_seg):

    cnn_seg[cnn_seg >= 0.5] = 1; cnn_seg[cnn_seg < 0.5] = 0

    struct = np.ones((3,3,3))

    pred_postp1 = cnn_seg

    labels, num_features = label(pred_postp1,struct)

    if num_features !=0:
        feature_size = np.zeros((num_features,))
        for ii in range(num_features):
            feature_size[ii] = np.sum(labels==(ii+1))

            label_liver = np.argmax(feature_size)
            pred_postp = np.zeros(cnn_seg.shape)
            pred_postp[labels== (label_liver+1)] = 1

    else:
        pred_postp = cnn_seg
    return pred_postp

In [118]:
all_paths = get_all_paths(study_folder)
print(len(all_paths))
print(all_paths[1])

691
N:\2_Analysis\automated_analysis\Training Cases\FLINT_CYNCH_SEGSEG\flint_output\2396---B17_Upgrade_Evolve_Camrd371 - 1---echo1.npy


In [119]:
echo5_paths = []

for root, dirs, files in os.walk(study_folder):
    for file in files:
        if file.endswith("echo5.npy"):
             echo5_paths.append(os.path.join(root, file))

In [120]:
print(len(echo5_paths))
echo5_paths[:5]

115


['N:\\2_Analysis\\automated_analysis\\Training Cases\\FLINT_CYNCH_SEGSEG\\flint_output\\2396---B17_Upgrade_Evolve_Camrd371 - 1---echo5.npy',
 'N:\\2_Analysis\\automated_analysis\\Training Cases\\FLINT_CYNCH_SEGSEG\\flint_output\\2413---B17_Upgrade_Evolve_Camrd371 - 1---echo5.npy',
 'N:\\2_Analysis\\automated_analysis\\Training Cases\\FLINT_CYNCH_SEGSEG\\flint_output\\2421---Mri_Duke_2421_Smu_092311_1026 - 1---echo5.npy',
 'N:\\2_Analysis\\automated_analysis\\Training Cases\\FLINT_CYNCH_SEGSEG\\flint_output\\2422---B17_Upgrade_Evolve_Camrd371 - 1---echo5.npy',
 'N:\\2_Analysis\\automated_analysis\\Training Cases\\FLINT_CYNCH_SEGSEG\\flint_output\\2423---Liver_Scan - 1---echo5.npy']

In [123]:
for i in echo5_paths:
    study_num = i[80:]
    if study_num.startswith('Cynch_'):
        study_num = study_num[6:]
        study_num = study_num[:4]
    else:
        study_num = study_num[:4]
        
    ni_path = r'N:\2_Analysis\automated_analysis\Training Cases\Flint\Images\Mag_echo5/' + 'f_' + study_num + '.nii'
    img_ni = nib.load(ni_path)
    
    npy_path = i
    npy_arr = np.load(npy_path)
    
    liv,seg = process_mask(npy_arr,img_ni.shape)
    
    liv = np.swapaxes(liv,0,1)
    liv = np.flip(liv,0)
    liv = np.flip(liv,1)
    seg = np.swapaxes(seg,0,1)
    seg = np.flip(seg,0)
    seg = np.flip(seg,1)
    
    seg = seg.astype('int8')
    seg_ni = nib.Nifti1Image(seg, img_ni.affine)
    liv_ni = nib.Nifti1Image(liv, img_ni.affine)
    
    nib.save(liv_ni, r'N:\2_Analysis\automated_analysis\Training Cases\FLINT_CYNCH_SEGSEG\flint_output_nii\segmentation/' + 'f_'+ study_num + '.nii')
    nib.save(seg_ni, r'N:\2_Analysis\automated_analysis\Training Cases\FLINT_CYNCH_SEGSEG\flint_output_nii\segmentalsegmentation/' + 'f_'+ study_num + '.nii')